## Overview

The Langmuir isotherm adsorption model is $\theta_1 = \frac{\sigma_1 \mu}{1 + \sigma_1 \mu}$ where $\mu = \frac{m}{M} = \frac{p}{\sqrt{2 \pi M R T}} = 43.75 \times 10^{-6} \frac{p}{\sqrt{M T}} \equiv$ number of gram molecules of gas striking each sq. cm. per second, $\sigma_1 = \frac{\alpha}{\nu_1} \equiv$ "relative life," and $\alpha \theta \mu = \nu_1 \theta_1 \equiv$ condensation and evaporation rates of gas in equilibrium with surface.[$^{[1]}$](#1)

With experimental measurements of $\sigma_1$ at different temperatures $T$ and pressures $p$, a Gaussian proccess (GP) model may be fitted using FoKL[$^{[2]}$](#2) such that $\theta_1$ is defined symbolically in Pyomo.[$^{[3,4]}$](#3)

## II. Experiments with Glass[$^{[1]}$](#1)

- Equation 32 yields $m x + b$
    - for FokL, maybe fit $(a, b)$ directly to $(p, q)$
        - won't work because linear no matter $(a, b)$; also $a = f(b)$ and $b = f(a)$
- under **Discussion of Results with Mica and Glass**, it seems the results were obtained without $\sigma_1$ being known; it seems $\theta_1$ was somehow measured
    - sort of, achieved by $m x + b$ Equation 32 by measuring only $(p, q_{obs})$ where $q_{obs}$ is simply $C_0 - C_e \equiv C_{initial} - C_{equilibrium}$
    - Equation 9 with $\theta_1 \equiv \frac{q}{q_{max}}$ yields $q = \frac{q_m \sigma_1 \mu}{1 + \sigma_1 \mu} \implies \frac{p}{q} = \frac{1}{a b} + \frac{p}{b}$ where $a p \equiv \sigma_1 \mu$ and $b \equiv q_m$
        - $\implies a p = 43.75 \times 10^{-6} \frac{\alpha p}{\nu_1 \sqrt{M T}}$
        - maybe... $\sigma_1 \equiv \frac{\alpha}{\nu_1} = \frac{\exp{(\Delta_f G / R T})}{\exp{(\Delta_b G / R T)}} = \exp{(\frac{\Delta_f G - \Delta_b G}{R T})}$
        - $\implies a = 43.75 \times 10^{-6} \exp{(\frac{\Delta_f G - \Delta_b G}{R T})} / \sqrt{M T}$

## Potential FoKL Method

Original Langmuir isotherm:

$\begin{cases}
\theta_1 = \frac{\sigma_1 \mu}{1 + \sigma_1 \mu} \equiv \text{adsorption model, } \mu \text{ known } f(T, p) \\
\sigma_1 = \frac{\alpha}{\nu_1} \equiv \text{relative life} \\
\theta_1 \equiv \frac{q}{q_{max}} \implies q = \frac{q_m \sigma_1 \mu}{1 + \sigma_1 \mu} = \frac{abp}{1+ap} \equiv \text{Equation 31} \\
\frac{p}{q} = \frac{1}{ab} + \frac{p}{b} \equiv \text{linearization of Equation 31 } (\frac{p}{q} \text{ vs. } p)
\end{cases}$

Instead of linearization for $q_{cal} = f(p)$ where $q_{obs}$ is measured at constant temperature $T$, attempting to train FoKL model $\mathcal{F}$ for $\theta_1$ or $q_{cal}$ at any $(T, p)$ pair. Training exponent to avoid negatives:

$\sigma_1 = \frac{\alpha}{\nu_1} = \exp{(\frac{\Delta_f G - \Delta_b G}{R T})} \\
\implies \mathcal{F} \equiv \Delta_f G - \Delta_b G = f(T, p)$

The novelty here is assuming $\Delta G = V dT - S dp$ does not work analytically in practice, so $\mathcal{F}$ would instead provide a symbolic equation for $\Delta_f G - \Delta_b G$.

Or, since previous work already results in $q_{cal} = f(p)$ at constant $T$, perhaps FoKL can provide $q_{cal} = f(T, p)$. In this case, simply training FoKL models $a = f(T)$ and $b = f(T)$ will allow $q_{cal} = f(T, p)$. However, it would be possible for this to result in negative values for $a$ and/or $b$ and thus $q$.

It is expected:
- $a \propto T^{-1/2} \exp{(T)}$
- $a^{2} \propto T^{-1} \exp{(2 T)}$
- $a^{-2} \propto T \exp{(-2 T)}$
- $\ln{(a^{-2})} \propto \ln{(T \exp{(-2 T)})} = \ln{(T)} - 2 T$

In this way, $\ln{(a^{-2})} + 2T$ may be trained on $\ln{(T)}$ such that $a$ is always positive. Note the *linear* dependence on $\ln{(T)}$ instead of $T^{-1/2} \exp{(T)}$ or variations is expected to drive higher accuracy. With FoKL model $\mathcal{F} \equiv \ln{(a^{-2})} + 2T \propto \ln{(T)}$,
- $a = 1 / \sqrt{\exp{(\mathcal{F} - 2T)}}$

Now, $b$ is simply $q_{max}$, which likely depends in a more complicated way on $(T, p)$, but $b$ is only necessary for $q$ and not $\theta_1$. For $\theta_1$,
- $\theta_1 = \frac{\sigma_1 \mu}{1 + \sigma_1 \mu} = \frac{ap}{1+ap} = \frac{p / \sqrt{\exp{(\mathcal{F} - 2T)}}}{1 + p / \sqrt{\exp{(\mathcal{F} - 2T)}}}$

Or, more simply,
- $\theta_1 = \frac{\sigma_1 \mu}{1 + \sigma_1 \mu} = \frac{ap}{1+ap} = \frac{p}{a^{-1}+p} = \frac{p}{p + \sqrt{\exp{(\mathcal{F} - 2T)}}}$

Or, furthermore,
- $\mathcal{Z} \equiv p^{-1} \sqrt{\exp{(\mathcal{F} - 2T)}}$
- $\theta_1 = \frac{1}{1 + \mathcal{Z}}$

## Reconstructing from ground-up

### Target
Langmuir isotherm site fraction $\theta_1$ in terms of temperature $T$ and pressure $p$

### Solution
Experimental results yield $[\theta_1 = f(a, p)]_{T=\text{const.}}$ where $a$ is a model parameter estimated from a linearization of the isotherm model. To enable $\theta_1 = f(T, p)$ instead, it was initially thought to train a FoKL model $\mathcal{F}_{GP}$ on $a \propto T$.

However, to prevent negative values of $\theta_1$ since $\mathcal{F}_{GP}$ is not bounded to positive values, training directly on $a$ is not desired. Also, to enable higher accuracy, the definition of $\mathcal{F}_{GP}$ was adjusted to avoid a nonlinear proportionality to its training input (i.e., $\ln(T)$). Satisfying both these requirements,

$\mathcal{F}_{GP} \equiv \ln{(a^{-2})} + 2T \propto \ln{(T)}$

Then, the site fraction is simply

$\theta_1 = (1 + \mathcal{Z})^{-1}$ where $\mathcal{Z} \equiv p^{-1} \sqrt{\exp{(\mathcal{F}_{GP} - 2T)}}$

However, only the mica experiment varies temperature, and it does so just twice at $a_{90^{\circ} \text{K}}$ and $a_{155^{\circ} \text{K}}$.

## References

<c id=1> [1] </c> [Langmuir, I. <br>
&emsp;&nbsp; The adsorption of gases on plane surfaces of glass, mica and platinum. <br>
&emsp;&nbsp; J. Am. Chem. Soc., 40 (1918), pp. 1361-1403.](https://pubs.acs.org/doi/epdf/10.1021/ja02242a004)

<c id=2> [2] </c> []()

<c id=3> [3] </c> []()

<c id=4> [4] </c> []()